In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install soynlp -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 6.9 MB/s eta 0:00:00


In [ ]:
from soynlp.word import WordExtractor
from soynlp import DoublespaceLineCorpus
from soynlp.noun import LRNounExtractor_v2
from soynlp.tokenizer import LTokenizer
import urllib.request
import re
import csv
import pickle
import numpy as np
import pandas as pd
from collections import Counter
from nltk.corpus import stopwords

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/02)2024-1학기/2_자연어처리/00_개인 프로젝트 논문 (웹툰굿즈)/데이터 및 코드/★라벨링_통합본.xlsx')
df = df.sort_values(by='brand')
df

,brand,product_name,review,rating
2094,가비지타임,2024 아크릴 캘린더,너무너무 구ㅏ여ㅜ어요… 진짜 구ㅏ여움….,1
2140,가비지타임,2024 아크릴 캘린더,넘 귀여워요 생각보다 작지만 아크릴이 귀여워서 괜차나여,1
2139,가비지타임,2024 아크릴 캘린더,생각보다 많이 작았지만 그래도 귀엽고 예뻐요..\n뒷면 필름 제거 하실때 조심하세요...,1
2138,가비지타임,2024 아크릴 캘린더,야. 오른쪽으로 가다가 멈춰서 아크릴 캘린더. 막아봐.,1
2137,가비지타임,2024 아크릴 캘린더,지금까지 나온 갑타 굿즈 중에 제일 가격이 합리적이라고 느껴지는 굿즈였습니다. 퀄리...,1
...,...,...,...,...
16572,화산귀환,만개화 디오라마 아크릴 스탠드,"2개 시켰는데 안구정화 제대롭니다.. \n구화산러나 최애가 청문, 청명인 분들은 꼭...",1
16573,화산귀환,만개화 디오라마 아크릴 스탠드,너무 맘에 들어요 겨울도 사야겠어요,1
16574,화산귀환,만개화 디오라마 아크릴 스탠드,조립이 다소 힘들었지만 너무 예뻐요,0
16565,화산귀환,만개화 디오라마 아크릴 스탠드,최고입니둥.ㅋㅋ 청명이 달려~,1


In [ ]:
df1 = df[df['rating']==1]
df1 = df1[df1['product_name']=='그립톡 (2종)']
df1 #마루는 강쥐_그립톡 (2종)_긍정 리뷰

,brand,product_name,review,rating
9526,마루는강쥐,그립톡 (2종),마루가 귀여운건 당연한거고 ㅋㅋ 그립톡이 진짜 짱짱하고 튼튼해요!! 옆으로 세워놓아...,1
9527,마루는강쥐,그립톡 (2종),아이가 너무 좋아하네요\n아까워서 쓰기싫대요ㅋ,1
9528,마루는강쥐,그립톡 (2종),"품절이 풀릴때까지 계속 기다렸던,,!! 인간마루 그립톡,,!! 실물 너무 귀여워요,...",1
9532,마루는강쥐,그립톡 (2종),귀엽습니다.. 친구 선물인데 아까워서 아직 사용은 못했다네요ㅠ 다음엔 강쥐마루도 하...,1
9535,마루는강쥐,그립톡 (2종),이미 GS 콜라보 그립톡 가지고 있는데 그것보단 퀄리티가 훨씬 좋네용 맥세이프용 그...,1
...,...,...,...,...
10491,마루는강쥐,그립톡 (2종),귀여운 마루.. 그립톡 첨인데 튼튼하니 좋아요!!,1
10492,마루는강쥐,그립톡 (2종),귀엽고 생각보다 튼튼해요,1
10493,마루는강쥐,그립톡 (2종),귀여워서 보자마자 붙여버렸어요. 급하게 붙이느라 붙이는 부분에 써도 되는 스티커 못...,1
10494,마루는강쥐,그립톡 (2종),기절 그냥 기절 마루야행복해..,1


In [ ]:
df0 = df[df['rating']==0]
df0 = df0[df0['product_name']=='그립톡 (2종)']
df0 #마루는 강쥐_그립톡 (2종)_부정 리뷰

,brand,product_name,review,rating
9529,마루는강쥐,그립톡 (2종),일단 마루가 너무 귀여워서 만족입니다!\n그렇지만 내구성은 좀 떨어지긴 하는듯......,0
9530,마루는강쥐,그립톡 (2종),일단 마루가 너무 귀엽고요!! 그래서 만족합니다.\n다만 쪼금 조심해서 써야할 것 ...,0
9531,마루는강쥐,그립톡 (2종),마루가 약간 창백하긴 하지만 그래도 귀여워요 ㅋㅋㅋ\n그립톡도 튼튼해요! 미니 엽서...,0
9525,마루는강쥐,그립톡 (2종),인간 마루만 사 봤는데 역시 강쥐 마루도 살 걸 그랬다~~!!~ 넘 기여버용♡ 다음...,0
9534,마루는강쥐,그립톡 (2종),두번째구매에여 이번에는 친구주려고삿어요\n하도오랴품절이라 오랫동안 기다렷내여\n재고...,0
...,...,...,...,...
10508,마루는강쥐,그립톡 (2종),"마루 사랑훼,, 모죠님 임티 빨리 내주세요\n텍마머니,,아임 레디,,",0
10510,마루는강쥐,그립톡 (2종),마루 ㅠㅠㅠ진짜 너무 귀여워요 ..보고 계시다면 이모티콘도 만들어주세요 🙏🏻,0
10484,마루는강쥐,그립톡 (2종),마루 짱짱!! 개귀여워요~~ 그립톡 첨써보는데 좋네요~ 비싸지만 마루가 귀여우니까 ...,0
10486,마루는강쥐,그립톡 (2종),인간 마루보다 살짝 작은 느낌이에요~~ 인간 마루 강쥐 마루 두 개 샀는데 매일 바...,0


In [ ]:
#긍정 전처리
df1["cut_special_character"]=df1['review'].apply(lambda x: re.sub(r"[^ ㄱ-ㅣ가-힣A-Za-z0-9]", " ", x))
df1["cut_korean_consonant"]=df1["cut_special_character"].apply(lambda x: re.sub(r"([ㄱ-ㅎㅏ-ㅣ]+)", " ", x))
df1["fiterd_data"]=df1["cut_korean_consonant"].apply(lambda x: re.sub(r"[a-z]([A-Z])", r"-\1", x).upper())

#부정 전처리
df0["cut_special_character"]=df0['review'].apply(lambda x: re.sub(r"[^ ㄱ-ㅣ가-힣A-Za-z0-9]", " ", x))
df0["cut_korean_consonant"]=df0["cut_special_character"].apply(lambda x: re.sub(r"([ㄱ-ㅎㅏ-ㅣ]+)", " ", x))
df0["fiterd_data"]=df0["cut_korean_consonant"].apply(lambda x: re.sub(r"[a-z]([A-Z])", r"-\1", x).upper())

In [ ]:
df1['fiterd_data'][0:10] #긍정

9526    마루가 귀여운건 당연한거고   그립톡이 진짜 짱짱하고 튼튼해요   옆으로 세워놓아도...
9527                             아이가 너무 좋아하네요 아까워서 쓰기싫대요 
9528    품절이 풀릴때까지 계속 기다렸던     인간마루 그립톡     실물 너무 귀여워요 ...
9532    귀엽습니다   친구 선물인데 아까워서 아직 사용은 못했다네요  다음엔 강쥐마루도 하...
9535    이미 GS 콜라보 그립톡 가지고 있는데 그것보단 퀄리티가 훨씬 좋네용 맥세이프용 그...
9536    이미 GS 콜라보 그립톡 가지고 있는데 그것보단 퀄리티가 훨씬 좋네용 맥세이프용 그...
9539    매우 많이 심각하도록 귀엽습니다  다음에는 꼭 마루 강아지 버전도 사리라 마음먹으며...
9533    생일선물로 준비했는데 친구가 굉장히 만족했습니다   튼튼해보이고 귀여워용 근데 인간...
9540                                   아직 사용 안햇지만 넘 귀여워요 
9522                                     좋아요 다음에도 구입하겠습니다
Name: fiterd_data, dtype: object

In [ ]:
df0['fiterd_data'][0:10] #부정

9529    일단 마루가 너무 귀여워서 만족입니다  그렇지만 내구성은 좀 떨어지긴 하는듯    ...
9530    일단 마루가 너무 귀엽고요   그래서 만족합니다  다만 쪼금 조심해서 써야할 것 같아요 
9531    마루가 약간 창백하긴 하지만 그래도 귀여워요   그립톡도 튼튼해요  미니 엽서같은 ...
9525    인간 마루만 사 봤는데 역시 강쥐 마루도 살 걸 그랬다      넘 기여버용  다음...
9534    두번째구매에여 이번에는 친구주려고삿어요 하도오랴품절이라 오랫동안 기다렷내여 재고확보...
9537         딸아이 선물로 사줬는데 완전 만족해요   금액대가 좀 있지만 좋아하니 짱입니다 
9538    혹시 반품된 물건을 다시 보내시는 건가요  오늘 받은 새 제품인데 윗 부분이 깨져서...
9524    강쥐마루 상품에서 불량이 있어서 교환 한번 하는데 3주가 걸리네요  택배측에서 알아...
9523               인간마루를 갖고싶었지만 품절이여서 강쥐마루로 주문했는데 너무귀여워요 
9520    귀엽고 만족합니다 만  별개로 좀비딸그립톡 구매했는데 오배송되서 반품신청함 단순오배...
Name: fiterd_data, dtype: object

In [ ]:
def listToString(str_list):
    result = ""
    for s in str_list:
        result += s + " "
    return result.strip()

In [ ]:
review1 = listToString(df1['fiterd_data'])
review1 #긍정

'마루가 귀여운건 당연한거고   그립톡이 진짜 짱짱하고 튼튼해요   옆으로 세워놓아도 중심을 잘 잡아서 너무 잘 샀더고 생각해여  아이가 너무 좋아하네요 아까워서 쓰기싫대요  품절이 풀릴때까지 계속 기다렸던     인간마루 그립톡     실물 너무 귀여워요   투명케이스에 붙이면 금방 때탈까봐 실리콘 케이스도 주문했답니닿  귀엽습니다   친구 선물인데 아까워서 아직 사용은 못했다네요  다음엔 강쥐마루도 하나 주려구요  이미 GS 콜라보 그립톡 가지고 있는데 그것보단 퀄리티가 훨씬 좋네용 맥세이프용 그립톡 사서 바꿔서 껴보려구요 이미 GS 콜라보 그립톡 가지고 있는데 그것보단 퀄리티가 훨씬 좋네용 맥세이프용 그립톡 사서 바꿔서 껴보려구요 매우 많이 심각하도록 귀엽습니다  다음에는 꼭 마루 강아지 버전도 사리라 마음먹으며 매일 하루하루 꼭 핸드폰을 뒤를 보며 마루를 보는 시간을 따로 가질 정도로 귀여워요   생일선물로 준비했는데 친구가 굉장히 만족했습니다   튼튼해보이고 귀여워용 근데 인간 마루 사주니까 강쥐 마루도 사주고 싶어요 아직 사용 안햇지만 넘 귀여워요  좋아요 다음에도 구입하겠습니다 너무 귀여워요  잘 떨어지지도 않고 접착력 강해서 더 좋아요  그립톡 두개 다 샀는데 둘다 사길 잘 한거 같아요   넘넘 기여워용 마루는 강쥐 팝업 갔을때 못 봐가지구 못 샀는데 온라인에 풀려서 바로 샀어요   마루 그립톡 넘 귀여워요 생각보다 사이즈가 더 커서 더 귀여워요 배송 빨라서 좋고 무엇보가 그립톡 편해서 좋아요 잘때 스탠드에 세워놓고 자면 안넘어지고 아주 잘있어요 하    이건 진짜 어케 쓰지 아까워서   마루그립톡 만들어주셔서 감사해요     마루가 너무 귀엽네요 아주 만족합니다  동생이 마루 귀엽다고 아주 좋아해요 배송 빠르고 상품질도 좋네요 귀여운 마루랑 함께 해서 GOOD 너무 귀엽고 튼튼하고 좋아요 너무 귀엽고 튼튼해요 여자친구가 좋아하네요    여워서 저도사려구요 귀야워서 또 재구매했어요  엄청 튼튼해요 너무 귀여워요 그립톡 정품이라서 튼튼하고 좋아요 

In [ ]:
review0 = listToString(df0['fiterd_data'])
review0 #부정

'일단 마루가 너무 귀여워서 만족입니다  그렇지만 내구성은 좀 떨어지긴 하는듯    참고하세요  일단 마루가 너무 귀엽고요   그래서 만족합니다  다만 쪼금 조심해서 써야할 것 같아요  마루가 약간 창백하긴 하지만 그래도 귀여워요   그립톡도 튼튼해요  미니 엽서같은 것도 구성품인데 이게 특히 귀엽네요   잘 쓰겠습니다   인간 마루만 사 봤는데 역시 강쥐 마루도 살 걸 그랬다      넘 기여버용  다음엔 으르렁 표정이나 그런 거도 나왔으면 좋겠네요  두번째구매에여 이번에는 친구주려고삿어요 하도오랴품절이라 오랫동안 기다렷내여 재고확보 마니좀해쥬새여 마루의귀여움은 강주 나 인간이나 똑같네야 딸아이 선물로 사줬는데 완전 만족해요   금액대가 좀 있지만 좋아하니 짱입니다  혹시 반품된 물건을 다시 보내시는 건가요  오늘 받은 새 제품인데 윗 부분이 깨져서 왔네요 혹시 배송 중에 깨진 건가 싶어서 박스 안을 봤는데 깨진 조각은 없었어요 그립톡 자체에는 문제가 없어서 구매 확정 누르긴 했지만 물건 보낼 때 확인하시고 보내 주세요 강쥐마루 상품에서 불량이 있어서 교환 한번 하는데 3주가 걸리네요  택배측에서 알아서 수거한다는 것만 알려주고 그 뒤부턴 순차적으로 답변을 준다고 되어있지만 톡톡을 보내도 무응답인 점이 실망스러웠어요  이러한 대응 방법은 달라졌으면 좋겠습니다  다행히 교환된 상품은 불량은 없네요  잘 쓰겠습니다   인간마루를 갖고싶었지만 품절이여서 강쥐마루로 주문했는데 너무귀여워요  귀엽고 만족합니다 만  별개로 좀비딸그립톡 구매했는데 오배송되서 반품신청함 단순오배송아니고 A주문했는데 물건없었는지 B를보내고 판매창에 품절때려버림 이거에 기분이 너무나빠서 씩씩거리고있는데 3일째 대답없고 기재된 전번은 걍 적어둔거임 절대안받음 톡톡도씹어서 페이챗상담했더니 확인중이니기다리라함 물건일부러잘못보내놓고 뭘그렇게확인할께있음  상품은 만족하지만 두번다시는 웹툰프렌즈 물건 구입 안할것임 귀여워요   사람마루도 갖고 싶네요   귀여워요 원래 있던 접착부분이 떨어져서 쓰리엠테이

#**kobart**

1. 라이브러리 및 모델 임포트
2. 긍정 리뷰 요약:
- 입력 텍스트를 토크나이저로 인코딩하여 토큰 ID로 변환합니다.
- 모델이 처리할 수 있는 최대 길이(max_input_length)를 512로 설정합니다.
- 입력 텍스트를 최대 길이에 맞게 여러 세그먼트로 나누어 처리합니다
- 각각의 입력 세그먼트에 대해 요약을 생성한 후 리스트에 추가
- 생성된 요약들을 하나로 합쳐 최종 요약을 생성
3. 부정 리뷰 요약:
- 위의 긍정 리뷰 요약 과정과 동일한 절차

In [ ]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
print("긍정 리뷰 길이: ",len(review1))
print("부정 리뷰 길이: ",len(review0))

긍정 리뷰 길이:  68444
부정 리뷰 길이:  18989


In [ ]:
#긍정 리뷰 요약
text = review1

# 입력 텍스트를 토크나이저로 인코딩하여 토큰 ID로 변환
raw_input_ids = tokenizer.encode(text)

max_input_length = 512  # 모델이 처리할 수 있는 최대 길이

# 입력 텍스트를 최대 길이에 맞게 여러 세그먼트로 나누어 처리
input_segments = []
for i in range(0, len(raw_input_ids), max_input_length):
    input_segments.append(raw_input_ids[i:i + max_input_length])

# 각각의 입력 세그먼트에 대해 요약 생성 후 리스트에 추가
summary_texts = []
for segment in input_segments:
    input_ids = [tokenizer.bos_token_id] + segment + [tokenizer.eos_token_id]
    summary_ids = model.generate(torch.tensor([input_ids]), num_beams=4, max_length=30, eos_token_id=1)
    summary_text = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
    summary_texts.append(summary_text)

# 생성된 요약들을 하나로 합쳐 최종 요약 생성
final_summary_1 = " ".join(summary_texts)
final_summary_1

' 절절이 풀릴때까지 계속 기다렸던    인간마루 그립톡이 진짜 짱짱하고 튼튼 요새 유행하는 마루와 강쥐 넘 귀엽네요  귀여워서 재구매했어요 마루가 너무 좋아해서 꼭 사고싶었는데 팝업을 못 가서 우울했다가 인터넷에 팔아서 헐레벌떡 사버렸습니다  마루 너무 귀엽고 튼튼해서 돈이 안아까워요         완전 튼튼하고 귀여버요 마루를야 사랑해 귀엽고 예쁘고 사랑스럽다 아이가 넘 마음에 들어하네요    귀여워윰    마루 최애 캐릭터에요        너무 귀엽고 튼튼해여 아주 원하던 그림입니다  폰케이스 안에 끼우고 그립톡 붙이기  우리 마루 넘 귀여워        폰에 사용은 마루강쥐 살앙해 너무 귀여움 접착력도 훌륭 퀄리치가 너무 좋아서 더 귀여운것 같아요    강쥐마루는 울집댕이 배신하는거 같아서 그냥 인간마루로 했어요          제발 인간마루 제 품에 넣고 싶네용   제발 인간마루 제 품에 넣고 싶네용   마루는 너무 귀엽고 귀엽고 귀여워요 그립톡 생각보다 크고 스크래치도 잘 안 나게 되어있 아이럽 마루그립톡 붙이려고 마루옷인 노란색케이스까지 샀는데 너무너무 귀엽고 튼튼해서 강쥐마루 인간마루 인간마루는 너무너무 귀엽고 사랑스러워서 꺄물고 싶지만 간신히 짱귀 대존귀 맨들맨들 귀여워요 너무너무 귀여워요 너무너무 귀여워요 너무너무 마루야 사랑해 내가 우리언닌 아니지만  함께 행복하자 정말 너무너무너무너무너무 귀여워요  제빵공장 네웹에서 제빵공장 네웹     마루 인형 출시를 기원합니다     강쥐마루 남친 줄려고 샀는데 너무 귀여워서 칭구들한테 자랑하고 부러움받고 다닙니다    마루는야 사랑해 너무 귀여워요  0 배송 생각보다 늦지 않았네요  0 마룽 하이룽 마룽 너무너 강주도 커엽  둘다 구매햇는데 일코는 강쥐버젼 더쿠는 휴멘 버젼하면될듯해여 강쥐마루가 드디어 배송왔어요          투명 케잇 에 마루를 븉였읍니다 마루 귀여워 귀여워 귀여워 귀여워 귀여워 귀여워 귀여워 귀여워 마루 마루 마셔서 감사합니다  마루 귀여워 마루 귀여워 마루 귀여워 마루 귀여

In [ ]:
#긍정 리뷰 요약
text = review0

# 입력 텍스트를 토크나이저로 인코딩하여 토큰 ID로 변환
raw_input_ids = tokenizer.encode(text)

max_input_length = 512  # 모델이 처리할 수 있는 최대 길이

# 입력 텍스트를 최대 길이에 맞게 여러 세그먼트로 나누어 처리
input_segments = []
for i in range(0, len(raw_input_ids), max_input_length):
    input_segments.append(raw_input_ids[i:i + max_input_length])

# 각각의 입력 세그먼트에 대해 요약 생성 후 리스트에 추가
summary_texts = []
for segment in input_segments:
    input_ids = [tokenizer.bos_token_id] + segment + [tokenizer.eos_token_id]
    summary_ids = model.generate(torch.tensor([input_ids]), num_beams=4, max_length=30, eos_token_id=1)
    summary_text = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
    summary_texts.append(summary_text)

# 생성된 요약들을 하나로 합쳐 최종 요약 생성
final_summary_0 = " ".join(summary_texts)
final_summary_0

'인간 마루만 사 봤는데 역시 강쥐 마루도 살 걸 그랬다     넘 기여버용  다음엔 마루 최고    너무 귀엽네요   마루 최고    너무 귀엽네요 분리가 되는거였으면 마루 폰케이스도 만들어주세여 조금 헐렁한 느낌이 없지않아있지만 귀여우니깐 봐줌 인간마루 강쥐마루는 1단계까지 가능인데 마루는 불량이지만 그냥 교환귀찮아서 씁니다       마루 이모티콘을 구매했는데 구매후에 부착하려니까 뭔가 덜컹거리는 느낌이 나더라고요    그립톡 마루짱 내 최애웹툰   이모티콘도 얼른 나와라  마루짱 마루는 강쥐 검색하니까 마루는 제 사랑이자 힐링이에요          마루는 제 사랑이자 힐링이에요    강쥐마루 왕귀엽다 짖짜     인간마루도 구매하고 싶었는데 품절이라 너무 아쉬 귀욥다증말   기다려서 받은 보람이 있구나 작가님 굿즈를 다양하게 만들어 주세오 마루야 너 진짜 마루 팬인데 너무 이뻐요         3 임티랑 굿즈 많이 나왔으면 좋겠어요  그립톡에 붙어있는 양면테이프 접착력은 그렇게 센 편은 아닌것같아요     마루 마루는강쥐 넘 즐거보고있는데 사길잘했어요    제발 마루 굿즈 더 많이 내주세요  실물 너무 귀엽고 생각보다 커여 정품이라 그런지 살짝 붙였는데 안떨어져짐 위치 잘잡으세요   마루와 늘 함께해서 넘 좋아욤 아이폰 미니는 딱 가운데 끼우면 각도가 안나와서 살짝 사이드로 끼워야 맞 인간마루가 품절돼서 아쉽지만 그냥 마루도 귀여워여 그 사이에 폰을 플립으로 바꿨는데 플립에 품절이어서 못 샀다가 입고 된거 보자마자 질렀는데 계속 살 수 있게 해주세요 제발  너무귀여움 높이 진짜킹왕짱지대기여븜  대왕그립톡이 올줄알았는데 생각보단 작아서 넘귀야 조네마루가 내폰에 그립톡 비싸고 내구성은 아직까지 좋은듯 귀여운건 크게보는게 옳다  생각보다 사람 마루 사고 싶었는데 품절이었어서 강쥐 마루로 했어요  근데 링이 없는 크립톡으로 변경했 인간 마루 강쥐 마루 강쥐 마루 둘 다 넘 깨물어주고싶어요  가격이 너무 사악하긴하지만요 마루 그립톡 종류대로 1개씩 사서 인간마루

In [ ]:
print("긍정 리뷰 요약 길이: ",len(final_summary_1))
print("부정 리뷰 요약 길이: ",len(final_summary_0))

긍정 리뷰 요약 길이:  3557
부정 리뷰 요약 길이:  1022
